In [2]:
import rasterio
import numpy as np
import pandas as pd
import geopandas as gpd

In [3]:
# Load the TIFF file
tiff_path = "D:\\MINE 590 - Nadja\\Project\\raster\\topo.tif"
with rasterio.open(tiff_path) as src:
    elevation = src.read(1)  # Read the first band (elevation data)
    transform = src.transform  # Get affine transformation to convert pixels to coordinates

# Get the number of rows and columns
rows, cols = elevation.shape

# Create empty lists to store data
x_coords, y_coords, elev_values = [], [], []

# Loop through each pixel
for row in range(rows):
    for col in range(cols):
        x, y = transform * (col, row)  # Convert pixel indices to coordinates
        x_coords.append(x)
        y_coords.append(y)
        elev_values.append(elevation[row, col])

# Print a sample (first 5 points)
for i in range(5):
    print(f"X: {x_coords[i]}, Y: {y_coords[i]}, Elev: {elev_values[i]}")

X: 501059.0019, Y: 5449033.8958, Elev: -3.4028234663852886e+38
X: 501060.0019019992, Y: 5449033.8958, Elev: -3.4028234663852886e+38
X: 501061.0019039983, Y: 5449033.8958, Elev: -3.4028234663852886e+38
X: 501062.0019059975, Y: 5449033.8958, Elev: -3.4028234663852886e+38
X: 501063.0019079967, Y: 5449033.8958, Elev: -3.4028234663852886e+38


In [25]:
# Create DataFrame
df = pd.DataFrame({"X": x_coords, "Y": y_coords, "Elev": elev_values})

# Filter: Keep only values between 0 and 120
df = df[(df["Elev"] >= 0) & (df["Elev"] <= 120)].copy()

# Check if filtering worked
print(df["Elev"].describe())  # Summary statistics to verify min/max values

# Now apply the absolute value transformation
df["Elevation"] = df["Elev"].abs() + 100
df["Slice"] = 1

count    5.829086e+06
mean     3.699687e+00
std      1.320271e+00
min      0.000000e+00
25%      3.429942e+00
50%      3.889917e+00
75%      4.240000e+00
max      1.354873e+01
Name: Elev, dtype: float64
                   X             Y  Elev  Elevation  Slice
9801   505958.011694  5.449033e+06   1.0      101.0      1
9802   505959.011696  5.449033e+06   1.0      101.0      1
14701  505956.011690  5.449032e+06   1.0      101.0      1
14702  505957.011692  5.449032e+06   1.0      101.0      1
14703  505958.011694  5.449032e+06   1.0      101.0      1


In [28]:
print(df.head())  # Print first few rows to verify

# Select only required columns
selected_columns = ["X", "Y", "Elevation", "Slice"]

print(df[selected_columns].head())  # Shows first 5 rows


                   X             Y  Elev  Elevation  Slice
9801   505958.011694  5.449033e+06   1.0      101.0      1
9802   505959.011696  5.449033e+06   1.0      101.0      1
14701  505956.011690  5.449032e+06   1.0      101.0      1
14702  505957.011692  5.449032e+06   1.0      101.0      1
14703  505958.011694  5.449032e+06   1.0      101.0      1
                   X             Y  Elevation  Slice
9801   505958.011694  5.449033e+06      101.0      1
9802   505959.011696  5.449033e+06      101.0      1
14701  505956.011690  5.449032e+06      101.0      1
14702  505957.011692  5.449032e+06      101.0      1
14703  505958.011694  5.449032e+06      101.0      1


In [29]:
# Export to a .txt file (tab-separated)
df[selected_columns].to_csv("D:\\MINE 590 - Nadja\\Project\\csv\\topo100m.txt", sep="\t", index=False, header=True)

In [30]:
output_path = "D:\\MINE 590 - Nadja\\Project\\csv\\topo.dat"

# Export to ASCII Database format (.dat) for FEFLOW
df[selected_columns].to_csv(output_path, sep="\t", index=False, header=True)

In [31]:
print("Minimum Elevation:", df["Elevation"].min())
print("Maximum Elevation:", df["Elevation"].max())


Minimum Elevation: 100.0
Maximum Elevation: 113.5487289428711
